# Import dependencies

In [58]:
import json
import pandas as pd
import numpy as np

# Read data

In [88]:
# Reading text from txt file
with open('../data/transactions.txt') as txt_file:
    lines = txt_file.readlines()

# Text in text file was in JSON format
# Converting text lines to JSON format and then restructuring as a dataframe 
trxn_data = []
for line in lines:
    trxn_data.append(json.loads(line))

# Replacing blank fields with NaN values
df = pd.DataFrame(trxn_data)
df = df.replace(r'', np.NaN)
print('Data has been successfully loaded.')

Data has been successfully loaded.


C:\Users\gorba\AppData\Local\Temp\ipykernel_21828\727062690.py:13: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace(r'', np.NaN)


In [89]:
df.head()

,accountNumber,customerId,creditLimit,availableMoney,transactionDateTime,transactionAmount,merchantName,acqCountry,merchantCountryCode,posEntryMode,...,echoBuffer,currentBalance,merchantCity,merchantState,merchantZip,cardPresent,posOnPremises,recurringAuthInd,expirationDateKeyInMatch,isFraud
0,737265056,737265056,5000.0,5000.0,2016-08-13T14:27:32,98.55,Uber,US,US,02,...,NaN,0.0,NaN,NaN,NaN,False,NaN,NaN,False,False
1,737265056,737265056,5000.0,5000.0,2016-10-11T05:05:54,74.51,AMC #191138,US,US,09,...,NaN,0.0,NaN,NaN,NaN,True,NaN,NaN,False,False
2,737265056,737265056,5000.0,5000.0,2016-11-08T09:18:39,7.47,Play Store,US,US,09,...,NaN,0.0,NaN,NaN,NaN,False,NaN,NaN,False,False
3,737265056,737265056,5000.0,5000.0,2016-12-10T02:14:50,7.47,Play Store,US,US,09,...,NaN,0.0,NaN,NaN,NaN,False,NaN,NaN,False,False
4,830329091,830329091,5000.0,5000.0,2016-03-24T21:04:46,71.18,Tim Hortons #947751,US,US,02,...,NaN,0.0,NaN,NaN,NaN,True,NaN,NaN,False,False


In [90]:
# function to find out base info about dataset
def simple_eda(df):
    total_na = df.isna().sum().sum()
    col_name = df.columns
    dtypes = df.dtypes
    uniq = df.nunique()
    na_val = df.isna().sum()
    duplicate_indices = df[df.duplicated()].index
    print('Dimensions:', df.shape[0], ' - rows', df.shape[1], ' - columns')
    print('Total NA\'s', total_na)
    if len(duplicate_indices) > 0:
        print('Duplicate rows indeces: ', duplicate_indices)
    else:
        print('There are no duplicates in this dataset!') 
    print("%38s %10s     %10s %10s" % ("Column Name", "Data Type", "Count Distinct", "NA Values"))
    for i in range(len(df.columns)):
        print("%38s %10s     %10s %10s" % (col_name[i], dtypes[i], uniq[i], na_val[i]))

In [91]:
simple_eda(df)

Dimensions: 786363  - rows 29  - columns
Total NA's 4728625
There are no duplicates in this dataset!
                           Column Name  Data Type     Count Distinct  NA Values
                         accountNumber     object           5000          0
                            customerId     object           5000          0
                           creditLimit    float64             10          0
                        availableMoney    float64         521916          0
                   transactionDateTime     object         776637          0
                     transactionAmount    float64          66038          0
                          merchantName     object           2490          0
                            acqCountry     object              4       4562
                   merchantCountryCode     object              4        724
                          posEntryMode     object              5       4054
                      posConditionCode     object          

C:\Users\gorba\AppData\Local\Temp\ipykernel_21828\1163208193.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print("%38s %10s     %10s %10s" % (col_name[i], dtypes[i], uniq[i], na_val[i]))


### Dataset Column Descriptions:

1. **`accountNumber`**:  
   The account number related to the transaction.

2. **`customerId`**:  
   A unique identifier for the customer.

3. **`creditLimit`**:  
   The credit limit of the customer’s account, indicating the maximum amount they are allowed to spend.

4. **`availableMoney`**:  
   The available funds on the customer’s account at the time of the transaction.

5. **`transactionDateTime`**:  
   The date and time when the transaction occurred.

6. **`transactionAmount`**:  
   The amount of money involved in the transaction.

7. **`merchantName`**:  
   The name of the merchant where the transaction took place.

8. **`acqCountry`**:  
   The country where the payment instrument (e.g., card) was issued.

9. **`merchantCountryCode`**:  
   The country code of the merchant’s location.

10. **`posEntryMode`**:  
    Describes how the card details were entered at the point of sale (e.g., chip, magnetic stripe, manual entry).

11. **`posConditionCode`**:  
    The condition at the point of sale during the transaction (e.g., whether the card was physically present or not) ???

12. **`merchantCategoryCode`**:  
    A code that represents the category of the merchant’s business (e.g., restaurant, clothing store).

13. **`currentExpDate`**:  
    The current expiration date of the card involved in the transaction.

14. **`accountOpenDate`**:  
    The date when the customer’s account was opened.

15. **`dateOfLastAddressChange`**:  
    The last time the customer changed their address.

16. **`cardCVV`**:  
    The CVV code associated with the card used for the transaction.

17. **`enteredCVV`**:  
    The CVV code entered by the customer during the transaction.

18. **`cardLast4Digits`**:  
    The last 4 digits of the card used in the transaction. Used to identify the specific card in use.

19. **`transactionType`**:  
    The type of transaction (e.g., purchase, withdrawal, refund).

20. **`echoBuffer`**:  
    A technical field that might contain auxiliary information related to the transaction.

21. **`currentBalance`**:  
    The account balance after the transaction has been processed.

22. **`merchantCity`**:  
    The city where the merchant is located.

23. **`merchantState`**:  
    The state where the merchant is located (applicable mostly to U.S. transactions).

24. **`merchantZip`**:  
    The postal code of the merchant's location.

25. **`cardPresent`**:  
    Indicates whether the physical card was present during the transaction (`True`/`False`).

26. **`posOnPremises`**:  
    Specifies whether the transaction occurred on the merchant’s premises or remotely (e.g., online).

27. **`recurringAuthInd`**:  
    Indicates whether the transaction is part of a recurring authorization (e.g., subscription).

28. **`expirationDateKeyInMatch`**:  
    Indicates if the entered expiration date matches the card’s actual expiration date (`True`/`False`).

29. **`isFraud`**:  
    A label indicating whether the transaction is fraudulent (`True`/`False`). This is the target variable for  anomaly detection model.


## Drop empty/useless columns

In [92]:
df = df.drop(columns={'recurringAuthInd', 'posOnPremises', 'merchantZip', 'merchantState', 'merchantCity', 'echoBuffer'})
simple_eda(df)

Dimensions: 786363  - rows 23  - columns
Total NA's 10447
There are no duplicates in this dataset!
                           Column Name  Data Type     Count Distinct  NA Values
                         accountNumber     object           5000          0
                            customerId     object           5000          0
                           creditLimit    float64             10          0
                        availableMoney    float64         521916          0
                   transactionDateTime     object         776637          0
                     transactionAmount    float64          66038          0
                          merchantName     object           2490          0
                            acqCountry     object              4       4562
                   merchantCountryCode     object              4        724
                          posEntryMode     object              5       4054
                      posConditionCode     object            

C:\Users\gorba\AppData\Local\Temp\ipykernel_21828\1163208193.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print("%38s %10s     %10s %10s" % (col_name[i], dtypes[i], uniq[i], na_val[i]))


In [93]:
# On the surface it seems that column accountNumber equals customerId. 
# So if it is true - let`s drop one of them

if(len(df[df.accountNumber != df.customerId]) == 0):
    df = df.drop(columns={'customerId'})
    simple_eda(df)

Dimensions: 786363  - rows 22  - columns
Total NA's 10447
There are no duplicates in this dataset!
                           Column Name  Data Type     Count Distinct  NA Values
                         accountNumber     object           5000          0
                           creditLimit    float64             10          0
                        availableMoney    float64         521916          0
                   transactionDateTime     object         776637          0
                     transactionAmount    float64          66038          0
                          merchantName     object           2490          0
                            acqCountry     object              4       4562
                   merchantCountryCode     object              4        724
                          posEntryMode     object              5       4054
                      posConditionCode     object              3        409
                  merchantCategoryCode     object            

C:\Users\gorba\AppData\Local\Temp\ipykernel_21828\1163208193.py:17: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print("%38s %10s     %10s %10s" % (col_name[i], dtypes[i], uniq[i], na_val[i]))


## Cleaning/filling NA
I need to drop/fill NA before data splitting, feature engineering and modeling. But before cleaning NA i need to analyze columns with NA

In [66]:
df.acqCountry.unique()

array(['US', nan, 'CAN', 'MEX', 'PR'], dtype=object)

In [67]:
df.merchantCountryCode.unique()

array(['US', 'CAN', nan, 'PR', 'MEX'], dtype=object)

In [98]:
df

,accountNumber,creditLimit,availableMoney,transactionDateTime,transactionAmount,merchantName,acqCountry,merchantCountryCode,posEntryMode,posConditionCode,...,accountOpenDate,dateOfLastAddressChange,cardCVV,enteredCVV,cardLast4Digits,transactionType,currentBalance,cardPresent,expirationDateKeyInMatch,isFraud
0,737265056,5000.0,5000.00,2016-08-13T14:27:32,98.55,Uber,US,US,02,01,...,2015-03-14,2015-03-14,414,414,1803,PURCHASE,0.00,False,False,False
1,737265056,5000.0,5000.00,2016-10-11T05:05:54,74.51,AMC #191138,US,US,09,01,...,2015-03-14,2015-03-14,486,486,767,PURCHASE,0.00,True,False,False
2,737265056,5000.0,5000.00,2016-11-08T09:18:39,7.47,Play Store,US,US,09,01,...,2015-03-14,2015-03-14,486,486,767,PURCHASE,0.00,False,False,False
3,737265056,5000.0,5000.00,2016-12-10T02:14:50,7.47,Play Store,US,US,09,01,...,2015-03-14,2015-03-14,486,486,767,PURCHASE,0.00,False,False,False
4,830329091,5000.0,5000.00,2016-03-24T21:04:46,71.18,Tim Hortons #947751,US,US,02,01,...,2015-08-06,2015-08-06,885,885,3143,PURCHASE,0.00,True,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
786358,732852505,50000.0,48904.96,2016-12-22T18:44:12,119.92,Lyft,US,US,90,01,...,2012-08-23,2012-08-23,936,936,3783,PURCHASE,1095.04,False,False,False
786359,732852505,50000.0,48785.04,2016-12-25T16:20:34,18.89,hulu.com,US,US,09,01,...,2012-08-23,2012-08-23,939,939,3388,PURCHASE,1214.96,False,False,False
786360,732852505,50000.0,48766.15,2016-12-27T15:46:24,49.43,Lyft,US,US,02,01,...,2012-08-23,2012-08-23,936,936,3783,PURCHASE,1233.85,False,False,False
786361,732852505,50000.0,48716.72,2016-12-29T00:30:55,49.89,walmart.com,US,US,09,99,...,2012-08-23,2012-08-23,939,939,3388,PURCHASE,1283.28,False,False,False


In [64]:
# 1) ?? remember better to split data to train/test before or after preprocessing ??
# 2) ?? remember scaling and check for what purpose to use it
# 3) ?? investigate how to process different types of anomalies (point, context, group)
# 4) Clean/Fill NA
# 5)-1 Make some statistic data analyse, data distribution to find out nature of anomalies
# 5)-2 Investigate importance of each columns (maybe drop useless)

# Process categorical data
# Check correlation
# Check if data is imbalanced and fix it

# Model types
# Hybrid - cluster + classifier
# Ensemble